In [1]:
import os
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
import numpy as np
from seqeval.metrics import classification_report

c:\Users\satis\OneDrive\Desktop\MiniProjectsem_7\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATASET_FOLDER = os.path.join("../data", "ner_dataset")
MODEL_CHECKPOINT = "dmis-lab/biobert-base-cased-v1.1"
OUTPUT_MODEL_FOLDER = os.path.join("../models", "clinical-ner-model")

In [3]:
LABELS = ["CONDITION", "DRUG", "LAB_TEST", "VALUE", "OPERATOR", "PROCEDURE", "DEMOGRAPHIC"]
tag_names = ['O'] + [f'{prefix}-{tag}' for tag in LABELS for prefix in ['B', 'I']]
id2tag = {i: tag for i, tag in enumerate(tag_names)}
tag2id = {tag: i for i, tag in enumerate(tag_names)}

In [4]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [[tag_names[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[tag_names[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    report = classification_report(true_labels, true_predictions, output_dict=True)
    return {"precision": report["macro avg"]["precision"], "recall": report["macro avg"]["recall"], "f1": report["macro avg"]["f1-score"]}

In [5]:
processed_dataset = load_from_disk(DATASET_FOLDER)

In [6]:
print(f"Loading model and tokenizer from '{MODEL_CHECKPOINT}'...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_CHECKPOINT, num_labels=len(tag_names), id2label=id2tag, label2id=tag2id
)

Loading model and tokenizer from 'dmis-lab/biobert-base-cased-v1.1'...


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
training_args = TrainingArguments(
    output_dir=OUTPUT_MODEL_FOLDER,
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    push_to_hub=False,
    no_cuda=True,
)

c:\Users\satis\OneDrive\Desktop\MiniProjectsem_7\venv\Lib\site-packages\transformers\training_args.py:1619: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [8]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\satis\AppData\Local\Temp\ipykernel_19356\2756122195.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
trainer.train()

Step,Training Loss
50,0.613500
100,0.001500
150,0.001000
200,0.000800
250,0.000600
300,0.000600
350,0.000500
400,0.000500
450,0.000400
500,0.000400


TrainOutput(global_step=732, training_loss=0.0424511114840238, metrics={'train_runtime': 378.4027, 'train_samples_per_second': 3.869, 'train_steps_per_second': 1.934, 'total_flos': 12719297833200.0, 'train_loss': 0.0424511114840238, 'epoch': 3.0})

In [11]:
trainer.save_model(OUTPUT_MODEL_FOLDER)